In [1]:
import sqlite3
import fitz 
DB_PATH = r"C:\Valcare\AI_RAG\CODE\git_flask_pusg_pro\quizz_app\data\qa.db"

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
      cur = conn.cursor()
      cur.execute("DELETE from sqlite_sequence")

In [2]:
p = r"C:\Valcare\AI_RAG\CODE\git_flask_pusg_pro\quizz_app\data\Class 11_Phy01_Ch02.pdf"
def parse_pdf(p):
    doc = fitz.open(p)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

In [3]:
import json
from pydantic import BaseModel, Field
from google import genai
class Question(BaseModel):
    answer: str
    prompt: str
    question_type: str
    hint: str
    explanation: str
    choices: list[str] = Field(default_factory=list)
    difficulty_level: str  # 'easy', 'medium', 'hard'
    category: str

def generate_questions(text):
    system_prompt = (
        "You are an expert teacher creating quiz questions **strictly from a given textbook chapter**.\n"
        "The goal is to test conceptual understanding based only on the provided syllabus-aligned text. "
        "The goal is to test conceptual understanding **based only on the provided text**, but you may use external facts if they support the core concepts.\n\n"
      
        "Each question must follow this schema:\n\n"
        "- `answer`: The correct option (exact).\n"
        "- `prompt`: Clear, syllabus-aligned question.\n"
        "- `question_type`: Always 'multiple_choice'.\n"
        "- `hint`: Short hint related to the concept.\n"
        "- `explanation`: Why the correct option is right.\n"
        "- `choices`: Exactly 4 options (answer + 2 plausible distractors).\n"
        "- `difficulty_level`: Choose from 'easy', 'medium', 'hard'.\n"
        "- `category`: The **main concept keyword only**, directly derived from the textbook content. "
        "Use only high-level scientific topics like 'acceleration', 'gravity', 'velocity', 'relative motion'. "
        "**Avoid vague or structural categories** like 'introduction', 'summary', 'example', 'definition', 'background'.\n\n"
        
        "Strictly follow the syllabus context. You may use external knowledge only if it supports syllabus concepts."
        "Return ONLY a JSON array of objects matching this schema—no extra text."
    )
    user_prompt = f"generate 50 quiz questions using this content: {text}"
    client = genai.Client(vertexai=True, project="uday-452605", location="us-central1")
    response = client.models.generate_content(
                                          model="gemini-2.5-flash",
                                          contents=[system_prompt, user_prompt],
                                          config={
                                          "response_mime_type":"application/json",
                                          "response_schema":list[Question],}
                                          )
    try:
        questions = response.candidates[0].content.parts[0].text
        questions = json.loads(questions)
        return questions
    except:
        return []
  

In [4]:
text = parse_pdf(p)
reponse = generate_questions(text)

In [5]:
reponse

[{'answer': 'Change in position of an object with time',
  'prompt': 'According to the introduction, what is the fundamental definition of motion?',
  'question_type': 'multiple_choice',
  'hint': 'Recall the first paragraph of Chapter 2.1.',
  'explanation': "The text states: 'Motion is change in position of an object with time.'",
  'choices': ['Change in position of an object with time',
   'Change in velocity over time',
   'The rate at which an object covers distance',
   'The force applied to an object over a duration'],
  'difficulty_level': 'easy',
  'category': 'Motion'},
 {'answer': 'Its size is much smaller than the distance it moves in a reasonable duration of time',
  'prompt': "Under what condition can an object in motion be accurately treated as a 'point object'?",
  'question_type': 'multiple_choice',
  'hint': 'Consider the relative scale of the object and its displacement.',
  'explanation': "The text states: 'This approximation is valid so far as the size of the obje

In [6]:
def save_questions(reponse):
    if not reponse:
        print("No questions to store.")
        return

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    values = []
    for qa in reponse:
        print(qa)
        values.append((
            qa['answer'],
            qa['prompt'],
            qa['question_type'],
            qa['hint'],
            qa['explanation'],
            json.dumps(qa.get('choices', [])),
            qa.get('difficulty_level', 'medium'),
            qa.get('category', 'general')
        ))

    cur.executemany('''
        INSERT INTO questions (
            answer, prompt, question_type, hint, explanation, choices,
            difficulty_level, category
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', values)
    conn.commit()
    conn.close()
    

In [8]:
save_questions(reponse)

{'answer': 'Change in position of an object with time', 'prompt': 'According to the introduction, what is the fundamental definition of motion?', 'question_type': 'multiple_choice', 'hint': 'Recall the first paragraph of Chapter 2.1.', 'explanation': "The text states: 'Motion is change in position of an object with time.'", 'choices': ['Change in position of an object with time', 'Change in velocity over time', 'The rate at which an object covers distance', 'The force applied to an object over a duration'], 'difficulty_level': 'easy', 'category': 'Motion'}
{'answer': 'Its size is much smaller than the distance it moves in a reasonable duration of time', 'prompt': "Under what condition can an object in motion be accurately treated as a 'point object'?", 'question_type': 'multiple_choice', 'hint': 'Consider the relative scale of the object and its displacement.', 'explanation': "The text states: 'This approximation is valid so far as the size of the object is much smaller than the distan